In [1]:
import io
import sys
import json
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [2]:
def get_embedding(text):
    # The model.encode() method already returns a list of floats
    return model.encode(text, convert_to_tensor=False).tolist()

In [3]:
# Read JSON file from path
with open("../../../data/books_data_reviews.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

In [5]:
# counter
i = 0

# get time
import time
start_time = time.time()

# Update each document in the JSON data
for document in data:
    print("Processing book: ", i)
    # Extract fields with checks for None
    book_title = document.get("book_title", "") or ""
    description = document.get("description", "") or ""
    authors = document.get("authors", "") or ""
    categories = document.get("categories", "") or ""

    # Combine relevant text fields
    combined_text = book_title + " " + description + " " + authors + " " + categories

    # Add additional fields if needed
    # For example, you might want to include publisher and publishedYear

    

    # Process reviews if they exist
    reviews = document.get("reviews", []) or ""
    if len(reviews) > 0:
        for review in reviews:
            summary = review.get("summary", "") or ""
            text = review.get("text", "") or ""

            # Combine review summary and text
            review_text = summary + " " + text
            combined_text += " " + review_text
            # Create an embedding for the review text
            # review_embedding = get_embedding(review_text)

            # Add the review embedding to the review dictionary
            # review["vector"] = review_embedding

    # Create an embedding for the combined text
    document["vector"] = get_embedding(combined_text)
    i += 1

    print("Time elapsed: " + str(time.time() - start_time) + ". Estimated time remaining: " + str((time.time() - start_time) / i * (len(data) - i)/60) + " minutes")

Processing book:  0
Time elapsed: 0.042001962661743164. Estimated time remaining: 148.6890479207039 minutes
Processing book:  1
Time elapsed: 0.10552239418029785. Estimated time remaining: 186.77639640569686 minutes
Processing book:  2
Time elapsed: 0.16453027725219727. Estimated time remaining: 194.14664121468863 minutes
Processing book:  3
Time elapsed: 0.2325611114501953. Estimated time remaining: 205.81658363342285 minutes
Processing book:  4
Time elapsed: 0.2981564998626709. Estimated time remaining: 211.09380804777143 minutes
Processing book:  5
Time elapsed: 0.36365556716918945. Estimated time remaining: 214.55476432111527 minutes
Processing book:  6
Time elapsed: 0.38080358505249023. Estimated time remaining: 192.5750929866518 minutes
Processing book:  7
Time elapsed: 0.415804386138916. Estimated time remaining: 183.9899758299192 minutes
Processing book:  8
Time elapsed: 0.4735887050628662. Estimated time remaining: 186.27383891079162 minutes
Processing book:  9
Time elapsed: 0

KeyboardInterrupt: 

In [ ]:
# Save the modified data back to the JSON file
with open('../../../data/books_data_reviews_embeddings.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)